#### This code serves to test Algorithm 1 for the Iterative Domain Information Framework

In [1]:
import glob as glob
import math, random
import pandas as pd
from DatasetNames import get_dataset_class_name
from ExtraFunctions import *
from MultiClassPrediction import MultiClassPrediction, MultiClassImageTask
from SingleClassPrediction import SingleClassPrediction

In [1]:
DATASET_NAME = "OrganCMNIST"
NUMBER_OF_IMAGES = 100 # Total number of images to be used

source_directory = f"../Datasets/{DATASET_NAME}/test"  # Change this to your test directory path
_, image_paths = get_balanced_random_images(source_directory, n = NUMBER_OF_IMAGES)

random.seed(55)
random.shuffle(image_paths)
initial_image_paths = image_paths.copy()

dataset_classes_of_interest = get_dataset_class_name(DATASET_NAME)
init_classes_of_interest = list(dataset_classes_of_interest.values())

classes_of_interest_list = [init_classes_of_interest] * len(image_paths)

NameError: name 'get_balanced_random_images' is not defined

#### Playground

In [3]:
number_of_experiment = 1

model_list = [
    {
        "model_name": "gpt-4o",
        "model_type": "openai",
    },
    {
        "model_name": "gpt-4o-mini",
        "model_type": "openai",
    },
    # {
    #     "model_name": "claude-3-5-sonnet-20241022",
    #     "model_type": "claude",
    # }
]

In [ ]:
mcp = MultiClassPrediction(batch_size = 1, max_workers=2)
scp = SingleClassPrediction(batch_size = 1, max_workers=2)

exp_results = []

# Iterate over the number of experiments
for experiment in range(number_of_experiment):
    print(f"Experiment {experiment+1}/{number_of_experiment}")

    # Iterate over the models
    for model in model_list:
        model_name = model["model_name"]
        model_type = model["model_type"]

        print(f"Model: {model_name}")
        print(f"Model Type: {model_type}")

        # Update the model
        mcp.update_model(model_name)
        scp.update_model(model_name)

        # For each model, initialize the variables for the Algorithm
        n = len(init_classes_of_interest)
        k = 2
        threshold = 2
        number_of_classes_to_predict = math.ceil(n / k)
        image_paths = initial_image_paths

        while number_of_classes_to_predict >= threshold:

            # Store a dictionary of the image paths and their classes of interest
            image_paths_classes_dict = {i[0]: i[1] for i in zip(image_paths, classes_of_interest_list)}

            # print(f"Number of classes to predict: {number_of_classes_to_predict}")
            # Create the multiclass image tasks
            multiclass_image_tasks = create_multiclass_image_tasks(image_paths, classes_of_interest_list, number_of_classes_to_predict, 
                                                                   dictionary_of_interest=image_paths_classes_dict,
                                                                   not_of_interest=True)
            # print("multiclass_image_tasks", multiclass_image_tasks)

            

            # Prompt the model to predict "number_of_classes_to_predict" classes for each image
            rs = mcp.process_images(multiclass_image_tasks, model_type)

            # print("rs_of_interest: ", rs)

            # Set the predicted classes as the new classes of interest
            result_list, image_paths, classes_of_interest_list = extract_image_classes_from_multiclass_predictions(rs, number_of_classes_to_predict)

            # Create a second variation of the multiclass image tasks which are the classes that are not of interest
            # multiclass_image_tasks_not_of_interest = create_multiclass_image_tasks(image_paths, 
            #                                                                        classes_of_interest_list, 
            #                                                                        number_of_classes_to_predict, 
            #                                                                     #    consistency_sample=int(math.ceil(len(classes_of_interest_list[0]) / 2)),
            #                                                                        dictionary_of_interest=image_paths_classes_dict, 
            #                                                                        not_of_interest=True)
            
            # print("multiclass_image_tasks_not_of_interest", multiclass_image_tasks_not_of_interest)
            
            # Prompt the model to predict the classes that are not of interest
            # rs_not_of_interest = mcp.process_images(multiclass_image_tasks_not_of_interest, model_type)
            # result_list, image_paths, classes_of_interest_list = extract_image_classes_from_multiclass_predictions(rs_not_of_interest, number_of_classes_to_predict)

            image_paths_classes = [dataset_classes_of_interest[image_path.split("\\")[-2]] for image_path in image_paths]
            curr_pred_acc = compute_top_n_accuracy(image_paths_classes, classes_of_interest_list)

            # print(f"Predicted Classes: {classes_of_interest_list}")
            # print(f"Correct Classes: {image_paths_classes}")
            print(f"Accuracy: {curr_pred_acc}")

            log_experiment_results(dataset_classes_of_interest, exp_results, experiment, model_name, model_type, number_of_classes_to_predict, image_paths, classes_of_interest_list)

            # Update the number of classes to predict
            number_of_classes_to_predict = math.ceil(number_of_classes_to_predict / k)

        if rs:
            # Result from Multiclass Prediction
            result_list, _, _ = extract_image_classes_from_multiclass_predictions(rs)
        else:
            # Go to SingleTask result list directly
            result_list = create_singleclass_image_tasks(image_paths, classes_of_interest_list)

        # Predict the single classes for each image
        f_res = scp.process_images(result_list, model_type)
        correct_classes, pred_classes, image_paths = extract_single_classes(f_res, dataset_classes_of_interest)

        # Count the number of matches
        pred_result = count_matches(correct_classes, pred_classes)

        # print(f"Predicted Classes: {pred_classes}")
        # print(f"Correct Classes: {correct_classes}")
        print(f"Accuracy: {pred_result}")

        log_experiment_results(dataset_classes_of_interest, exp_results, experiment, model_name, model_type, number_of_classes_to_predict, image_paths, classes_of_interest_list)

Experiment 1/1
Model: gpt-4o
Model Type: openai


ERROR:root:Max retries (3) exceeded: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}


Batch processing failed: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}
Failed batch: ['../Datasets/OrganCMNIST/test\\7\\image_6070.png']


ERROR:root:Max retries (3) exceeded: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}


Batch processing failed: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}
Failed batch: ['../Datasets/OrganCMNIST/test\\5\\image_1414.png']


...

ERROR:root:Max retries (3) exceeded: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}


Batch processing failed: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}
Failed batch: ['../Datasets/OrganCMNIST/test\\0\\image_1294.png']


ERROR:root:Max retries (3) exceeded: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}


Batch processing failed: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}
Failed batch: ['../Datasets/OrganCMNIST/test\\9\\image_5515.png']


ERROR:root:Max retries (3) exceeded: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}
ERROR:root:Max retries (3) exceeded: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image has of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_format'}}


In [ ]:
x_sam = pd.DataFrame(exp_results)

In [ ]:
pd.DataFrame(exp_results).to_csv(f"{DATASET_NAME}_results_Algorithm1_EXP1.csv", index=False)